In [1]:
#Сдесь я сделал парсинг с сайта где есть таблица с данными , ее взял как пример. 
import sqlalchemy
import sqlite3
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine('sqlite://',echo=False)

url = 'http://adilet.zan.kz/rus/docs/V20T0004656'
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')
table = soup.find_all('table')[2]
rows = table.find_all('tr')[1:]
data = []

previousIndex = None
previousNorm = None
previousGroup = None
for row in rows:
    # print(row)
    td = row.find_all('td')
    if len(td) == 1:
        previousGroup = td[0].getText().strip()
    elif len(td) == 5:
        previousIndex = td[0].getText().strip()
        previousNorm = td[4].getText().strip()
        data.append({'index': previousIndex, 'name': td[1].getText().strip(), 'content': td[2].getText().strip(), 'measure': 'тонна', 'price_norm': previousNorm, 'group': previousGroup})

for item in data:
    print(item)
    print('\n')

{'index': '1', 'name': 'Аммиачная селитра', 'content': 'N-34,4', 'measure': 'тонна', 'price_norm': '36 500', 'group': 'Азотные удобрения'}


{'index': '2', 'name': 'Сульфат аммония', 'content': 'N-21, S-24', 'measure': 'тонна', 'price_norm': '35 000', 'group': 'Азотные удобрения'}


{'index': '3', 'name': 'Карбамид', 'content': 'N-46,2', 'measure': 'тонна', 'price_norm': '50 500', 'group': 'Азотные удобрения'}


{'index': '4', 'name': 'Жидкое удобрение КАС+', 'content': 'N-28-34, K2O-0,052, SO3-0,046, Fe-0,04', 'measure': 'тонна', 'price_norm': '51 785', 'group': 'N-46'}


{'index': '5', 'name': 'Нитрат аммония жидкий, марка Б', 'content': 'NH4NO3-52', 'measure': 'тонна', 'price_norm': '211\xa0160,5', 'group': 'N-46'}


{'index': '6', 'name': 'Минеральные удобрения суперфосфат марки "Б"', 'content': 'P2О5-15, К2О-2', 'measure': 'тонна', 'price_norm': '17\xa0531,3', 'group': 'Фосфорные удобрения'}


{'index': '7', 'name': 'Концентрат и мука фосфоритные Чилисайского месторождения', 'cont

In [2]:
data = pd.DataFrame(data)

In [3]:
data

,index,name,content,measure,price_norm,group
0,1,Аммиачная селитра,"N-34,4",тонна,36 500,Азотные удобрения
1,2,Сульфат аммония,"N-21, S-24",тонна,35 000,Азотные удобрения
2,3,Карбамид,"N-46,2",тонна,50 500,Азотные удобрения
3,4,Жидкое удобрение КАС+,"N-28-34, K2O-0,052, SO3-0,046, Fe-0,04",тонна,51 785,N-46
4,5,"Нитрат аммония жидкий, марка Б",NH4NO3-52,тонна,"211 160,5",N-46
...,...,...,...,...,...,...
466,490,Удобрение Muhtar 10.5.20+TE,"O.M -10, H+F-5, N-10, P-5, K-20, B-0,02, Fe-0,...",тонна,432 500,"N-6, P2O5-12, K2O-36, MgO-2, S-4"
467,491,Удобрение Muhtar 5.25.5+TE,"O.M -10, H+F-5, N-5, P-25, K-5, B-0,02, Fe-0,0...",тонна,477 500,"N-6, P2O5-12, K2O-36, MgO-2, S-4"
468,492,Удобрение Muhtar 5.5.25+TE,"O.M -10, H+F-5, N-5, P-5, K-25, B-0,02, Fe-0,0...",тонна,477 500,"N-6, P2O5-12, K2O-36, MgO-2, S-4"
469,493,Удобрение Muhtar ORG\K,"O.M -50, C-20, N-1, K-1, ph-5-7",тонна,275 000,"N-6, P2O5-12, K2O-36, MgO-2, S-4"


In [4]:
#import sqlalchemy
#import sqlite3
#import numpy as np
#import pandas as pd
#from sqlalchemy import create_engine

#file = 'Ozkan_Erkin.xlsx'
#output='output.xlsx'

#engine = create_engine('sqlite://',echo=False)
#df = pd.read_excel(file)
#df.to_sql('employees',engine,if_exists='replace',index=False)

In [ ]:
#Парсинг это извлечь данные , я их извлек и хотел подключить к базе MySQL
# То что я спарсил , экспортировал в xlsx файл
# xlsx хотел экспортировать в базу MySQL 
# но у меня зависло  при кодключений к локальному хосту
import xlrd
import MySQLdb

basexcel = xlrd.open_workbook('Ozkan_Erkin.xlsx')
sheet = basexcel.sheet_by_name("Sheet1")

database = MySQLdb.connect(host='localhost',user='Erkin',password='Catek0203',db='testpy')

cursor = database.cursor()

query = """INSERT INTO products (id_index, id_name, id_content, measure, price_norm, id_group) VALUES (%s, %s, %s, %s, %s, %s)"""

for r in range(0, sheet.nrows):
        index   = sheet.cell(r,0).value
        name    = sheet.cell(r,1).value
        content = sheet.cell(r,2).value
        measure = sheet.cell(r,3).value
        price_norm  = sheet.cell(r,4).value
        group  = sheet.cell(r,5).value


        values = (index, name, content, measure, price_norm, group)


        cursor.execute(query, values)